In [1]:
import pandas as pd
import duckdb
from utils.quack import Quack
pd.set_option('display.max_columns', 45)
con = duckdb.connect("data/nfl.duckdb")

In [2]:
Quack.query('select * from snaps limit 100')

,game_id,pfr_game_id,season,game_type,week,player,pfr_player_id,position,team,opponent,offense_snaps,offense_pct,defense_snaps,defense_pct,st_snaps,st_pct,.progress
0,2013_01_ARI_STL,201309080ram,2013,REG,1,Chris Williams,WillCh03,G,LA,ARI,67.0,1.00,0.0,0.0,5.0,0.17,True
1,2013_01_ARI_STL,201309080ram,2013,REG,1,Harvey Dahl,DahlHa20,G,LA,ARI,67.0,1.00,0.0,0.0,5.0,0.17,True
2,2013_01_ARI_STL,201309080ram,2013,REG,1,Jake Long,LongJa20,T,LA,ARI,67.0,1.00,0.0,0.0,0.0,0.00,True
3,2013_01_ARI_STL,201309080ram,2013,REG,1,Scott Wells,WellSc20,C,LA,ARI,67.0,1.00,0.0,0.0,0.0,0.00,True
4,2013_01_ARI_STL,201309080ram,2013,REG,1,Sam Bradford,BradSa00,QB,LA,ARI,67.0,1.00,0.0,0.0,0.0,0.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2013_01_ATL_NO,201309080nor,2013,REG,1,Lance Moore,MoorLa00,WR,NO,ATL,52.0,0.75,0.0,0.0,0.0,0.00,True
96,2013_01_ATL_NO,201309080nor,2013,REG,1,Marques Colston,ColsMa00,WR,NO,ATL,51.0,0.74,0.0,0.0,0.0,0.00,True
97,2013_01_ATL_NO,201309080nor,2013,REG,1,Kenny Stills,StilKe00,WR,NO,ATL,36.0,0.52,0.0,0.0,0.0,0.00,True
98,2013_01_ATL_NO,201309080nor,2013,REG,1,Jed Collins,CollJe01,FB,NO,ATL,33.0,0.48,0.0,0.0,0.0,0.00,True


In [3]:
with open("./data/queries/player_feature_eng.sql", "r") as f:
    sql = f.read()

df = con.execute(sql).fetchdf()


depth = Quack.query("""select depth.*, snps.""")
off = depth.loc[depth.formation=='Offense'].copy()
#off.loc[off.offense_pct > 0]
depth

In [4]:
context_cols = ['player_id', 'player_display_name','position',
]
game_context = ['week','season_type', 'season', 'opponent_team', 'recent_team', 'game_id', 'game_type']
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',]
rush_stat_cols = [
     'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'fantasy_points', 'fantasy_points_ppr',
       'offense_snaps','offense_pct'
]

depth_cols = ['season', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'player_display_name']


In [5]:

qb_stats_eng = [
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "passing_interceptions",
    "sacks_suffered",
    "sack_yards_lost",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_2pt_conversions",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
       #"completion_pct"
]

rush_stats_eng = [
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "fantasy_points",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
      # 'yards_per_carry',
]
receiver_stats_eng = [
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "receiving_air_yards",
    "receiving_yards_after_catch",
    "receiving_first_downs",
    "target_share",
    "air_yards_share",
    "fantasy_points_ppr",
      'offense_snaps','offense_pct'
]
stats_to_eng = list(set(qb_stats_eng + rush_stats_eng + receiver_stats_eng))

In [6]:
temp = df.copy().sort_values(by=['season','week']).reset_index(drop=True)

temp = off.merge(
    weekly,
    how="outer",
    left_on=["gsis_id", "team", "season", "week", "position", "game_id"],
    right_on=["player_id", "team", "season", "week", "position","game_id"],
).sort_values(by=['season','week']).reset_index(drop=True)
temp[context_cols] = temp.groupby(depth_cols)[context_cols].bfill()
temp[context_cols] = temp.groupby(depth_cols)[context_cols].ffill()


In [7]:
temp.loc[~temp.offense_snaps.isna(), stat_cols] = temp.loc[~temp.offense_snaps.isna(), stat_cols].fillna(0)
player_fantasy = temp.copy(deep=True).sort_values(by=['season','week']) 
roll_columns = [x + '_roll' for x in stats_to_eng]
player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[stats_to_eng].transform(lambda x: x.ewm(span=7, min_periods=0).mean().shift())
#player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[roll_columns].shift(1)

In [8]:
player_fantasy['completion_pct_roll'] = player_fantasy['completions_roll'] / player_fantasy['attempts_roll']
player_fantasy['yards_per_carry_roll'] = (3 + player_fantasy['rushing_yards_roll']) / (1 + player_fantasy['carries_roll'])
player_fantasy['yards_per_catch_roll'] = (4 + player_fantasy['receiving_yards_roll']) / (1 + player_fantasy['receptions_roll'])
player_fantasy['yards_per_target_roll'] = (7 + player_fantasy['receiving_yards_roll']) / (1 + player_fantasy['targets_roll'])
receiver_stats_eng += ['yards_per_catch','yards_per_target']
rush_stats_eng += ['yards_per_carry']
qb_stats_eng += ['completion_pct']

In [9]:
player_fantasy.loc[player_fantasy.player_display_name =='Brock Bowers'].sort_values(by=['season','week'])#[['season','week','offense_snaps', 'receptions', 'fantasy_points_ppr', 'yards_per_catch_roll', 'receptions_roll']]

,dense_depth,position,gsis_id,game_id,season,week,player_id,player_display_name,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,...,passing_air_yards_roll,sack_yards_lost_roll,passing_first_downs_roll,receiving_tds_roll,fantasy_points_ppr_roll,attempts_roll,receiving_yards_after_catch_roll,passing_yards_after_catch_roll,receiving_first_downs_roll,targets_roll,receptions_roll,sacks_suffered_roll,offense_pct_roll,rushing_tds_roll,carries_roll,rushing_first_downs_roll,air_yards_share_roll,receiving_yards_roll,completion_pct_roll,yards_per_carry_roll,yards_per_catch_roll,yards_per_target_roll
57890,0,TE,00-0039338,2024_01_LV_LAC,2024,1,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58467,0,TE,00-0039338,2024_02_LV_BAL,2024,2,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.000000,11.800000,0.0,16.000000,0.0,3.000000,8.000000,6.000000,0.0,0.680000,0.0,0.000000,0.000000,0.261745,58.000000,NaN,3.000000,8.857143,7.222222
58539,1,TE,00-0039338,2024_03_CAR_LV,2024,3,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.000000,15.800000,0.0,30.857143,0.0,4.142857,8.571429,7.714286,0.0,0.680000,0.0,0.000000,0.000000,0.248347,80.857143,NaN,3.000000,9.737705,9.179104
58865,0,TE,00-0039338,2024_04_CLE_LV,2024,4,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.000000,12.037838,0.0,27.459459,0.0,3.216216,6.594595,5.675676,0.0,0.671351,0.0,0.000000,0.000000,0.171348,63.621622,NaN,3.000000,10.129555,9.298932
59167,0,TE,00-0039338,2024_05_LV_DEN,2024,5,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.000000,9.500571,0.0,23.634286,0.0,2.405714,5.280000,4.331429,0.0,0.722057,0.0,0.365714,0.365714,0.143777,47.302857,NaN,5.410042,9.622722,8.646952
59578,0,TE,00-0039338,2024_06_PIT_LV,2024,6,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.327785,14.154930,0.0,36.537772,0.0,2.928297,7.482714,5.533931,0.0,0.744328,0.0,0.245839,0.245839,0.181978,63.592830,NaN,4.775951,10.344895,8.321962
59718,0,TE,00-0039338,2024_07_LV_LA,2024,7,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.228096,14.746481,0.0,39.111375,0.0,2.645976,8.248292,6.588061,0.0,0.758218,0.0,0.171072,0.171072,0.194359,65.845560,NaN,4.314735,9.204666,7.876650
60163,1,TE,00-0039338,2024_08_KC_LV,2024,8,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.162288,16.060224,0.0,40.810312,0.0,3.613651,9.907727,7.572445,0.0,0.781813,0.0,0.121716,0.121716,0.198584,73.679932,NaN,3.976578,9.061584,7.396585
60353,1,TE,00-0039338,2024_09_LV_CIN,2024,9,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.117202,14.598867,0.0,37.529258,0.0,3.443171,8.544298,6.857787,0.0,0.817425,0.0,0.087902,0.087902,0.201796,69.323849,NaN,3.727193,9.331361,7.996800
61006,0,TE,00-0039338,2024_11_LV_MIA,2024,11,00-0039338,Brock Bowers,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.355818,14.842439,0.0,32.791187,0.0,3.053089,8.397177,6.355637,0.0,0.839745,0.0,0.064142,0.064142,0.196683,62.749234,NaN,3.542484,9.074569,7.422360


In [10]:
player_fantasy =player_fantasy.groupby(['team','season','week','game_id', 'position', 'dense_depth', 'gsis_id']).head(1)

In [13]:
player_weekly_agg_raw = player_fantasy[[x for x in player_fantasy.columns if x not in stat_cols]]
player_weekly_agg_raw.to_parquet("data/agg/player_weekly_agg.parquet")
con.execute("CREATE OR REPLACE TABLE player_weekly_agg AS SELECT * FROM player_weekly_agg_raw")


In [14]:
agg = player_fantasy.copy()
agg = agg.sort_values(by=['team','season','week','player_display_name'])


In [15]:
def stack_players(df, num_players, index_cols, value_cols):
    'df should be seperated by position already'
    df = df.loc[df.dense_depth < num_players]
    piv = df.pivot(index=index_cols, columns=['position','dense_depth'], values=value_cols).reset_index()
    piv.columns = [x[0] + '_' + x[1] +'_' +  (str(int(x[2]))) if x[2] != '' else x[0] + x[1] for x in piv.columns ]
    return piv

In [16]:
dup_rows = agg[agg.duplicated(subset=['position','dense_depth'] + ['team','season','week'], keep=False)]
dup_rows = dup_rows.loc[dup_rows.position == 'WR']
dup_rows.loc[dup_rows.game_id == '2022_02_ATL_LA'][['gsis_id','game_id','player_display_name','dense_depth', 'position', 'team']].sort_values(by=['position','dense_depth'])

,gsis_id,game_id,player_display_name,dense_depth,position,team


In [17]:
test = df.loc[df.position == 'WR'].copy()
test.loc[test.game_id == '2022_02_ATL_LA'][['gsis_id','game_id','player_display_name','dense_depth', 'position', 'team']].sort_values(by=['team','dense_depth'])

,gsis_id,game_id,player_display_name,dense_depth,position,team
46951,00-0035208,2022_02_ATL_LA,Olamide Zaccheaus,0,WR,ATL
46965,00-0034854,2022_02_ATL_LA,KhaDarel Hodge,1,WR,ATL
46886,00-0036365,2022_02_ATL_LA,Bryan Edwards,3,WR,ATL
47145,00-0037238,2022_02_ATL_LA,Drake London,5,WR,ATL
47081,00-0031428,2022_02_ATL_LA,Allen Robinson,0,WR,LA
47009,00-0033908,2022_02_ATL_LA,Cooper Kupp,1,WR,LA
47173,00-0036862,2022_02_ATL_LA,Ben Skowronek,2,WR,LA
46979,00-0034646,2022_02_ATL_LA,Brandon Powell,3,WR,LA


In [18]:
cols_to_agg =  (['gsis_id','player_display_name'] + roll_columns)

In [19]:
qb_roll_stat_cols = ['gsis_id','player_display_name'] + [x + '_roll' for x in qb_stats_eng] 
receiver_roll_stat_cols = ['gsis_id','player_display_name'] + [x + '_roll' for x in receiver_stats_eng] 
rush_stat_cols = ['gsis_id','player_display_name'] + [x + '_roll' for x in rush_stats_eng]

team_qb_stats = stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team','season','week'], qb_roll_stat_cols)
wr_temp = stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team','season','week'], receiver_roll_stat_cols)
te_temp = stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],3,  ['team','season','week'], receiver_roll_stat_cols)
rb_temp =stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],2,  ['team','season','week'], receiver_roll_stat_cols)
team_receiver_stats = wr_temp.merge(te_temp, on=['team','season','week'])
team_receiver_stats = team_receiver_stats.merge(rb_temp, on=['team','season','week'])
team_rb_stats = stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],3,  ['team','season','week'], rush_stat_cols)

In [20]:
stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team','season','week'], receiver_roll_stat_cols)

,team,season,week,gsis_id_WR_2,gsis_id_WR_3,gsis_id_WR_0,gsis_id_WR_1,gsis_id_WR_4,player_display_name_WR_2,player_display_name_WR_3,player_display_name_WR_0,player_display_name_WR_1,player_display_name_WR_4,receptions_roll_WR_2,receptions_roll_WR_3,receptions_roll_WR_0,receptions_roll_WR_1,receptions_roll_WR_4,targets_roll_WR_2,targets_roll_WR_3,targets_roll_WR_0,targets_roll_WR_1,...,fantasy_points_ppr_roll_WR_1,fantasy_points_ppr_roll_WR_4,offense_snaps_roll_WR_2,offense_snaps_roll_WR_3,offense_snaps_roll_WR_0,offense_snaps_roll_WR_1,offense_snaps_roll_WR_4,offense_pct_roll_WR_2,offense_pct_roll_WR_3,offense_pct_roll_WR_0,offense_pct_roll_WR_1,offense_pct_roll_WR_4,yards_per_catch_roll_WR_2,yards_per_catch_roll_WR_3,yards_per_catch_roll_WR_0,yards_per_catch_roll_WR_1,yards_per_catch_roll_WR_4,yards_per_target_roll_WR_2,yards_per_target_roll_WR_3,yards_per_target_roll_WR_0,yards_per_target_roll_WR_1,yards_per_target_roll_WR_4
0,ARI,2013,1,00-0027691,00-0030300,00-0022921,00-0029638,NaN,Andre Roberts,Jaron Brown,Larry Fitzgerald,Michael Floyd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARI,2013,2,00-0027691,00-0030300,00-0022921,00-0029638,NaN,Andre Roberts,Jaron Brown,Larry Fitzgerald,Michael Floyd,NaN,8.0,1.0,8.0,4.0,NaN,9.0,2.0,14.0,6.0,...,12.2,NaN,51.0,12.0,71.0,67.0,NaN,0.72,0.17,1.0,0.94,NaN,11.222222,9.0,9.333333,17.2,NaN,10.4,7.0,5.8,12.714286,NaN
2,ARI,2013,3,00-0030300,00-0027691,00-0029638,00-0022921,NaN,Jaron Brown,Andre Roberts,Michael Floyd,Larry Fitzgerald,NaN,1.0,5.142857,3.428571,4.571429,NaN,2.0,8.428571,6.571429,8.857143,...,15.028571,NaN,12.571429,52.142857,67.0,56.714286,NaN,0.175714,0.737143,0.94,0.8,NaN,8.142857,10.767442,11.677419,10.25641,NaN,6.428571,7.333333,7.226415,6.101449,NaN
3,ARI,2013,4,00-0030300,00-0027691,00-0029638,00-0022921,NaN,Jaron Brown,Andre Roberts,Michael Floyd,Larry Fitzgerald,NaN,0.567568,3.351351,3.675676,4.756757,NaN,2.0,6.513514,6.324324,7.621622,...,13.459459,NaN,10.594595,43.0,61.810811,56.837838,NaN,0.16027,0.647568,0.944324,0.877838,NaN,7.0,9.621118,11.179191,10.741784,NaN,4.657658,5.971223,7.546125,7.520376,NaN
4,ARI,2013,5,00-0027691,NaN,00-0029638,00-0022921,NaN,Andre Roberts,NaN,Michael Floyd,Larry Fitzgerald,NaN,2.491429,NaN,4.16,5.211429,NaN,4.862857,NaN,7.668571,8.125714,...,15.412571,NaN,40.805714,NaN,58.222857,57.994286,NaN,0.630171,NaN,0.906171,0.911543,NaN,8.653028,NaN,12.874862,10.553818,NaN,5.664717,NaN,8.009888,7.51221,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6320,WAS,2024,20,00-0035659,NaN,00-0036626,NaN,00-0035208,Terry McLaurin,NaN,Dyami Brown,NaN,Olamide Zaccheaus,5.658107,NaN,3.347163,NaN,3.459913,8.587059,NaN,3.97719,NaN,...,NaN,9.70739,58.970543,NaN,37.311623,NaN,42.92904,0.834162,NaN,0.529366,NaN,0.604027,10.204197,NaN,10.559595,NaN,9.913119,7.399625,NaN,9.825682,NaN,8.280083
6321,WAS,2024,21,00-0036626,00-0035208,00-0035659,00-0039355,NaN,Dyami Brown,Olamide Zaccheaus,Terry McLaurin,Luke McCaffrey,NaN,4.010373,2.594935,5.24358,0.646685,NaN,4.982892,4.026375,7.940294,1.071774,...,1.185791,NaN,43.483719,40.44678,61.727907,18.324805,NaN,0.604524,0.56302,0.858121,0.247744,NaN,11.96083,9.501907,11.805002,5.70301,NaN,10.518026,7.392752,8.579749,5.980894,NaN
6322,WAS,2025,1,00-0033591,00-0039355,00-0035659,00-0035719,00-0040164,Noah Brown,Luke McCaffrey,Terry McLaurin,Deebo Samuel Sr.,Jaylin Lane,3.35038,0.482824,4.682685,3.314053,NaN,5.695357,1.053588,7.70522,5.022972,...,10.75556,NaN,44.894005,16.722187,63.29593,46.976833,NaN,0.641622,0.222977,0.858591,0.77679,NaN,9.91452,5.411999,12.14727,9.082884,NaN,6.890137,5.368674,8.274243,7.003859,NaN
6323,WAS,2025,2,00-0033591,00-0040164,00-0035659,00-0035719,00-0039355,Noah Brown,Jaylin Lane,Terry McLaurin,Deebo Samuel Sr.,Luke McCaffrey,3.012785,1.0,4.012014,4.23554,0.360896,5.0

In [21]:
team_qb_stats.to_csv('data/team_qb_stats.csv')
team_receiver_stats.to_csv('data/team_receiver_stats.csv')
team_rb_stats.to_csv('data/team_rb_stats.csv')

con.execute("CREATE OR REPLACE TABLE team_qb_stats AS SELECT * FROM team_qb_stats")
con.execute("CREATE OR REPLACE TABLE team_receiver_stats AS SELECT * FROM team_receiver_stats")
con.execute("CREATE OR REPLACE TABLE team_rb_stats AS SELECT * FROM team_rb_stats")

In [22]:
qb_rush_stats =stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team','season','week'], rush_stat_cols)
qb_rush_stats = qb_rush_stats.fillna(0)
te_rush_stats =stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],1,  ['team','season','week'], rush_stat_cols)
te_rush_stats = te_rush_stats.fillna(0)
wr_rush_stats =stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],1,  ['team','season','week'], rush_stat_cols)
wr_rush_stats = wr_rush_stats.fillna(0)

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_17988\1118549396.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  qb_rush_stats = qb_rush_stats.fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_17988\1118549396.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  te_rush_stats = te_rush_stats.fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_17988\1118549396.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in

In [24]:
temp = team_rb_stats.merge( qb_rush_stats,  on=['team','season','week'])
temp = temp.merge( te_rush_stats,  on=['team','season','week'])
temp = temp.merge(wr_rush_stats,  on=['team','season','week'])
temp.to_csv('data/team_rushers.csv')

con.execute("CREATE OR REPLACE TABLE team_rushers AS SELECT * FROM temp")

In [25]:
con.commit()
con.close()